# Train notebook - Airbus kaggle challenge

# Ship Detection- Project AIRBUS 2019
## Model Parameters
We might want to adjust these later (or do some hyperparameter optimizations)

In [1]:
BATCH_SIZE = 32
EDGE_CROP = 16
GAUSSIAN_NOISE = 0.1
UPSAMPLE_MODE = 'SIMPLE'
# downsampling inside the network
NET_SCALING = None
# downsampling in preprocessing
IMG_SCALING = (2,2)
# number of validation images to use
VALID_IMG_COUNT = 900
# maximum number of steps_per_epoch in training
MAX_TRAIN_STEPS = 10000
MAX_TRAIN_EPOCHS = 10

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
from preprocess.pre_process import multi_rle_encode, rle_encode, rle_decode, masks_as_image, masks_as_color, balancing_train
from preprocess.pre_process import make_image_gen, create_aug_gen
from sklearn.model_selection import train_test_split
from keras import backend as K

Using TensorFlow backend.


In [3]:
import tensorflow as tf
with tf.Session() as sess:
    devices = sess.list_devices()
for device in devices:
    print(device)
    
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 15859938834893636264)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 5367360497763996311)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 1811342145945570893)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 15560753152, 11486174805798916547)
Default GPU Device: /device:GPU:0


In [4]:
ship_dir = '../../data/airbus_ship_detection/'
train_image_dir = os.path.join(ship_dir, 'train')# Images for training
test_image_dir = os.path.join(ship_dir, 'test')# Images for testing
label_dir = os.path.join(ship_dir, 'train_ship_segmentations_v2.csv')# Images for testing
masks = pd.read_csv(label_dir) # Markers for ships

In [5]:
label_has_ships = masks
label_has_ships['has_ship'] = masks.EncodedPixels.notnull()
label_has_ships = label_has_ships.drop(columns=['EncodedPixels'])
label_has_ships = label_has_ships.drop_duplicates()

In [18]:
count_has_ship = label_has_ships.has_ship.sum()
count_has_ship
print("rate:", count_has_ship/len(label_has_ships))

rate: 0.2210058372629261


In [43]:
data_balanced = label_has_ships.groupby('has_ship')
data_balanced = data_balanced.apply(lambda x: x.sample(data_balanced.size().min()).reset_index(drop=True))
data_balanced

ImageId  has_ship
has_ship                               
False    0      4d6a4543e.jpg     False
         1      926614354.jpg     False
         2      58eb0e484.jpg     False
         3      4f8cd65de.jpg     False
         4      71fd5ad5f.jpg     False
         5      0ff2a91ad.jpg     False
         6      7b5c3caa8.jpg     False
         7      90b90029a.jpg     False
         8      41a338aa5.jpg     False
         9      4b3982121.jpg     False
         10     d40603666.jpg     False
         11     f0333434d.jpg     False
         12     4398eb5ac.jpg     False
         13     2bc428030.jpg     False
         14     802f8aadb.jpg     False
         15     67cbf55d9.jpg     False
         16     c9180aa77.jpg     False
         17     daba0655c.jpg     False
         18     0bbdb3071.jpg     False
         19     f157fc044.jpg     False
         20     016975974.jpg     False
         21     8ae518b3c.jpg     False
         22     8a4ee0f0f.jpg     False
         23     e56849487.jpg     False
         24     91eaf7676.jpg     False
         25     310da928e.jpg     False
         26     aa6e7a022.jpg     False
         27     3a6177d39.jpg     False
         28     c92894166.jpg     False
         29     b66db41af.jpg     False
...                       ...       ...
True     42526  4f16e3f9b.jpg      True
         42527  4346eebe9.jpg      True
         42528  c15528427.jpg      True
         42529  e4d809e33.jpg      True
         42530  a513a6051.jpg      True
         42531  de899dfa0.jpg      True
         42532  bfc2ab8b6.jpg      True
         42533  ee1445a09.jpg      True
         42534  dce2138f8.jpg      True
         42535  aa4088091.jpg      True
         42536  db967cb45.jpg      True
         42537  eb22eb36f.jpg      True
         42538  055ceb1c0.jpg      True
         42539  0ef56991d.jpg      True
         42540  f9daf268e.jpg      True
         42541  b5e0ad87c.jpg      True
         42542  4a42f5af7.jpg      True
         42543  fcf47fd84.jpg      True
         42544  8e48e5dcd.jpg      True
         42545  5e89141f4.jpg      True
         42546  7c27e9e3b.jpg      True
         42547  dd253c905.jpg      True
         42548  96c1d0f74.jpg      True
         42549  63b25cc3b.jpg      True
         42550  b6ffd0b57.jpg      True
         42551  adc547e14.jpg      True
         42552  dc2dc59ed.jpg      True
         42553  2420c86dc.jpg      True
         42554  3c810b73f.jpg      True
         42555  c200af2c5.jpg      True

[85112 rows x 2 columns]

In [44]:
count_has_ship = data_balanced.has_ship.sum()
count_has_ship
print("rate:", count_has_ship/len(data_balanced))

rate: 0.5


In [45]:
def make_image_gen(in_df, train_image_dir, batch_size=48, img_scalling=None):
    all_batches = list(in_df.groupby('ImageId'))
    out_rgb = []
    out_label = []
    while True:
        np.random.shuffle(all_batches)
        for c_img_id, c_label in all_batches:
            rgb_path = os.path.join(train_image_dir, c_img_id)
            c_img = imread(rgb_path)
            
            if img_scalling is not None:
                c_img = c_img[::img_scalling[0], ::img_scalling[1]]
                
            out_rgb.append(c_img)
            out_label.append(c_label.has_ship.values[0])
            if len(out_rgb)>=batch_size:
                yield np.stack(out_rgb, 0)/255.0, np.array(out_label)
                out_rgb, out_label=[], []   

In [46]:
training_set, validation_set = train_test_split(data_balanced, test_size=0.05)
print(len(training_set))
print(len(validation_set))

80856
4256


In [37]:
train_gen = make_image_gen(training_set, train_image_dir, BATCH_SIZE, IMG_SCALING)
train_x, train_y = next(train_gen)
print('x', train_x.shape, train_x.min(), train_x.max())
print('y', train_y.shape, train_x.min(), train_x.max())

x (32, 384, 384, 3) 0.0 1.0
y (32,) 0.0 1.0


In [47]:
train_y

array([False, False, False, False,  True, False, False, False, False,
       False, False, False, False,  True,  True, False, False, False,
       False, False, False, False, False,  True, False, False, False,
       False, False, False, False,  True])

In [48]:
import keras.backend as K
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from models.resnet50_classif import get_resnet50_classif

In [49]:
model = get_resnet50_classif(input_shape=train_x.shape[1:])
model.summary()

/usr/local/lib/python3.5/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 384, 384, 3)       0         
_________________________________________________________________
resnet50 (Model)             multiple                  23587712  
_________________________________________________________________
flatten_2 (Flatten)          (None, 294912)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               37748864  
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 61,336,705
Trainable params: 61,283,585
Non-trainable params: 53,120
_________________________________________________________________


In [50]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard, Callback
weight_path = "../weights_models/{}_weights.best.hdf5".format('model_1')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min', save_weights_only=True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                   patience=2, verbose=1, mode='min',
                                   min_delta=0.001, cooldown=1, min_lr=1e-7)

early = EarlyStopping(monitor="val_loss", mode="min", verbose=2,
                      patience=2) # probably needs to be more patient, but kaggle time is limited

tensorboard =  TensorBoard(log_dir="../logs/log1", update_freq='batch')

callbacks_list = [reduceLROnPlat, early]

In [51]:
#run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

model.compile(optimizer=Adam(), loss=binary_crossentropy, 
              metrics=["accuracy"])

step_count_train = min(MAX_TRAIN_STEPS, training_set.shape[0]//BATCH_SIZE)
training_gen = make_image_gen(training_set, train_image_dir, BATCH_SIZE, IMG_SCALING)
#training_aug_gen =  create_aug_gen(training_gen, image_gen, label_gen)

step_count_valid = validation_set.shape[0]//BATCH_SIZE
validation_gen = make_image_gen(validation_set, train_image_dir, BATCH_SIZE, IMG_SCALING)

print("step_count_train =", step_count_train)
print("step_count_valid =", step_count_valid)


loss_history = [model.fit_generator(training_gen,
                                 steps_per_epoch=step_count_train,
                                 epochs=MAX_TRAIN_EPOCHS,
                                 callbacks=callbacks_list,
                                 validation_data=validation_gen,
                                 validation_steps=step_count_valid)]

step_count_train = 2526
step_count_valid = 133
Epoch 1/10
 366/2526 [===>..........................] - ETA: 1:13:31 - loss: 1.3697 - acc: 0.5009

KeyboardInterrupt: 